In [1]:
# !pip install requests
# !pip install beautifulsoup4
# !pip install wikipedia
# !pip install PyDictionary
# !pip install pytrends
# !pip install urbandictionary

In [2]:
import pytrends
from pytrends.request import TrendReq
import wikipedia
import requests
from bs4 import BeautifulSoup
import random
import PyDictionary
from PyDictionary import PyDictionary
#import spacy
import pandas as pd
import re

dir(pytrends)
pytrends = TrendReq(hl='en-US', tz=360) # create a pytrends object
#pytrends.trending_searches(pn='united_states') # trending searches in real time for United States
# create a dictionary of the top 1000 trending searches for this year in the United States (as of 2022)
trending_searches_dict = pytrends.top_charts(
    date='2022',    # specifies the time period as the last year
    geo='US',            # specifies the country as the United States
)
real_time_trending_searches_dict = pytrends.trending_searches(pn='united_states') # trending searches in real time for United States



# show how many trending searches there are
print(f'Retrieved {len(trending_searches_dict)} trending searches.')
# show the top 10 trending searches in real time for United States
trending_searches_dict.head(10)

Retrieved 10 trending searches.


,title,exploreQuery
0,Wordle,
1,Election results,
2,Betty White,
3,Queen Elizabeth,
4,Bob Saget,
5,Ukraine,
6,Mega Millions,
7,Powerball numbers,
8,Anne Heche,
9,Jeffrey Dahmer,


In [3]:
import urbandictionary as ud
import re
# adding import alias recommended
bad_patterns = [r'sex*', r'porn*',r'fuck*',r'-ass*','ass','shit',r'damn*',r'ass|asse*',r'cock*',r'whor*',r'nigg*',r'slut*','blowjob',r'fagg*',r'boob|boob*', r'bitch*', r'bastard*',' ho |hoe',r'breast*|jugs', r'cunt*', r'puss*', r'dick*', 'naked', r'nud*', r'masterb*',r'mastu',r'nipple*',r'penis|penal|peni*','god','jesus','christ','bible','church','religion','pray','prayer','faith','lord','allah','muslim','islam','allah','ejaculate','jew*','islamic','atheist',r'rapist*|rape*',r'pedo*','atheism','atheists','atheist','atheists','christian','christianity','christians','christian','christians','gay',r'tit*|titt*','jesus','christ','bible','church','religion','pray','prayer','faith','lord','allah','muslim','islam','allah','islamic','atheist','atheism','atheists','atheist','atheists','christian','christianity','christians','christian','christians','yahwey','yeshua',r'israel*',r'sex*', r'porn*',r'fuck*',r'-ass*','ass','shit',r'damn*',r'ass|asse*',r'cock*',r'whor*',r'nigg*',r'slut*','blowjob','bj',r'fagg*',r'boob|boob*', r'breast*|jugs', r'cunt*', r'puss*', r'dick*', 'naked', r'nud*', r'nipple*',r'penis|penal|peni*','jesus','christ','bible','church','religion','pray','prayer','faith','lord','gay',r'tit*|titt*', 'fellatio', 'fuck', 'nigger','lynch',r'erotic*','screw','lay','right wing','black man','black girl','nigga','racial']

# to make the game more fun we can replace words with less inflamatory ones.


import re

import nltk

def check_for_badwords(definition, bad_patterns):
    # if any of the buzzwords are found return a list of the matches, else an empty list
    # include porter stemmer to match variations of the words in the bad_patterns list
    from nltk.stem import PorterStemmer
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in bad_patterns if len(stemmer.stem(word)) >= 4]
    # only include the stemmed words which begin with the same letter as the original word (this is to avoid adding words that are not related to the original word)
    stemmed_words = [word for word in stemmed_words if word[0] == word[0]]
    bad_patterns.extend(stemmed_words)
    # only include stemmed words that are not already in the list of words
    bad_patterns = list(set(bad_patterns))
    # create a list of regex patterns that match the words in the list
    patterns = [re.compile(rf'\b{word}\b', re.IGNORECASE) for word in bad_patterns]
    # return the list of regex patterns (regex patterns are used to match words, and are more efficient than using the 'in' operator)
    
    matches = [match.group(0) for match in re.finditer(r'\b(?:' + '|'.join(bad_patterns) + r')\b', definition, re.IGNORECASE)]
    if len(matches)>0:
        print(f'Found bad words in definition: {matches}')
    return matches


def check_for_good_patterns(definition, title):
    # check both title and definition for good patterns
    good_patterns = [r'\b\w+phobia\.?\b', r'\bslang\b', r'\bacronymn\b', r'\bmeme\b']


    if any(re.match(r'\b' + word + r'\b', definition) for word in good_patterns):
        print(f'\nFound a good pattern in the definition: {definition}')
        return True
    elif any(re.match(r'\b' + word + r'\b', title) for word in good_patterns):
        print(f'Found a good pattern in the title: {title}')
        return True
    else:
        return False

def remove_undesireable_sentences(definition, title, bad_patterns):
    # convert the definition and title to lowercase
    definition = definition.lower()
    title = title.lower()

    # split the definition into sentences
    sentences = definition.split('.')

    # split the title into words
    title_words = title.split()

    # initialize a list to store the acceptable sentences
    acceptable_sentences = []

    # iterate over the sentences in the definition
    for sentence in sentences:
        # split the sentence into words
        sentence_words = sentence.split()

        # check if any of the words in the sentence are also in the title
        if not any(word in title_words for word in sentence_words):
            # if not, add the sentence to the list of acceptable sentences
            acceptable_sentences.append(sentence)

    # remove sentences that are not in English
    definition = re.sub(r'[^\x00-\x7f]',r'', '.'.join(acceptable_sentences))
    # remove sentences that contain a regex match to any word in the buzzwords list
    definition = re.sub(r'|'.join(map(re.escape, bad_patterns)), '', definition)
    return definition

replacements = {
    r'\bsex\b': 'affection',
    r'\bporn\w*\b': 'adult entertainment',
    r'fuck': 'have intimate relations with',
    r'fucking': 'romancing, physically',
    r'\bfuck\w*\b': ' (bleep) ',
    r'\bass\b': 'dump truck',
    r'\bshit\w*\b': 'poop',
    r'damn\w*\b': 'darn',
    r'god-damn\w*\b|goddamn\w*\b': 'super darn',
    r'ass|asse*': 'rear end',
    r'cock': 'rooster',
    r'small dick energy':'little man syndrome',
    r'\bdick\w*\b': ' johnson ',
    r'ppl':'people',
    'wat':'what',
    r'a black': 'an african-american',
    r'black person':'dark skinned person',
    r'pee' : 'urinate',
    r'piss': 'urinate',
    r'\b(peeing|pissing)': 'urinating',
    r'white person':'caucasian',
    r'hole': ' cave ',
    r'\bwhore\w*\b': 'prostitute',
    r'nigg*': 'racial slur',
    '\r\n':' ',
    r'slut': ' loose person ',
    r'blowjob': 'random gift',
    r'racist': 'ignorant',
    'racism':'fear of people that look different',
    r'faggot': 'gay person',
    r'\bfag\w*\b':'gay person',
    r'boob|boob*|breast': 'lady pillows',
    r'\bbitch*\b': 'mean woman',
    r'bastard*': 'illegitimate child',
    r'hoes?': 'chicks',
    r'breast*|jugs': 'chest',
    r'\bcunt*\b': 'comtemptible person',
    r'\bpuss\w*\b': 'vagina',
    r'\wdick*': 'penis',
    r'naked': 'disrobed',
    r'\bnud\w*\b': 'unclothed',
    r'\nmasterbate\w*\b': 'self-gratified',
    r'\bmasturbating\w*\b': 'gratifying themselves',
    r'\b\w{4}ilf\b': 'person id like to get to know',
    r'mastu\w*\b': 'self-gratification',
    'god':'deity',
    'jesus':'religious figure',
    'christ':'religious figure',
    'bible':'religious text',
    'church':'place of worship',
    'religion':'set of beliefs',
    r'\bpray\w*\b':'communicate with a deity',
    'prayer':'a conversation with a deity',
    'faith':'belief',
    ' lord ':'captain',
    ' allah ':'a diety',
    'gay':'homosexual',
    r' rapist ': ' intense toucher ',
    r' rape ': ' unwelcomed tickling ', # eek I know... 
    r'pedo\w*\b': 'seventies mustached van driver',
    'yahwey':'a deity',
    'yeshua':'a religious figure',
    ' sexual':' reproductive',
    'douche':'chad',
    ' sex ':' private adult time ',
    'milf':"mother I would like to take on a date",
    ' butt ':"dumptruck/tailgate"
}

def replacer_censor(definition,phrase,replacements_dict):
    # Iterate over the keys in the replacements dictionary
    for pattern in replacements_dict:
        # Use re.sub to replace the occurrences of the pattern with its corresponding value in the phrase and definition strings
        phrase = re.sub(pattern, replacements_dict[pattern], phrase)
        definition = re.sub(pattern, replacements_dict[pattern], definition)
    return phrase, definition

def unpack_definitions(phrase,definition):
    # remove the brackets and clean up the definitions
    # with regex
    definition = definition.replace("[","")
    definition = definition.replace("]","")
    definition = definition.replace("'","")
    definition = definition.replace('"',"")
    definition = definition.replace("(","")
    definition = definition.replace(")","")

    # remove double spaces
    definition = definition.replace('  ',' ')
    # also remove any .. by replacing them with a single .
    definition = definition.replace('..','.')
    # remove all non-ascii characters
    definition = re.sub(r'[^\x00-\x7f]',r'', definition)
    # remove any words that are not in the english dictionary
    #english_words = set(w.lower() for w in nltk.corpus.words.words())
    #definition = re.sub(r'\b\w+\b', lambda m: m.group(0) if m.group(0) in english_words else '', definition)
    # remove extra spaces
    #definition = re.sub(' +', ' ', definition)
    
    phrase, definition = replacer_censor(definition,phrase,replacements)
    

    return phrase, definition

# import wikipedia

# def get_page_length(phrase):
#   # search for pages on Wikipedia that match the given phrase
#     try:
#         pages = wikipedia.search(phrase)

#         # retrieve the first page from the search results
#         page = wikipedia.page(pages[0])
#         print(f'Found the page for {page.title}', end='')
#         if phrase.lower() != page.title.lower():
#             print(' ... nevermind... not the right page.')
#             return 0
#         # return the length of the page
#         return len(page.content)
#     except Exception as e:
#             return 0



In [4]:
# def prepper(words):
#     # create a list of regex patterns that match the words in the list
#     patterns = [re.compile(rf'\b{word}\b', re.IGNORECASE) for word in words]

#     # add any variations of the words to the list of words by porter stemming
#     from nltk.stem import PorterStemmer
#     stemmer = PorterStemmer()
#     stemmed_words = [stemmer.stem(word) for word in words if len(stemmer.stem(word)) >= 4]
#     # only include the stemmed words which begin with the same letter as the original word (this is to avoid adding words that are not related to the original word)
#     stemmed_words = [word for word in stemmed_words if word[0] == word[0]]
#     words.extend(stemmed_words)
#     # only include stemmed words that are not already in the list of words
#     words = list(set(words))
#     # create a list of regex patterns that match the words in the list
#     patterns = [re.compile(rf'\b{word}\b', re.IGNORECASE) for word in words]
#     # return the list of regex patterns (regex patterns are used to match words, and are more efficient than using the 'in' operator)
#     return patterns



# # create a function that takes a list of words and a list of regex patterns and returns a list of words that match the regex patterns
# def match_words(words):
#     words = words.split() # split the words string into a list of words
#     try:
#         patterns = prepper(words) # create the regex patterns
#         matched_words = [] # initialize a list to store the matched words (words that match the regex patterns and indicate a bad word)
#         for pattern in patterns: # iterate over the patterns (regex patterns)
#             for word in words: # iterate over the words in the list of words.
#                 if pattern.match(word): # if the word matches the regex pattern, add it to the list of matched words, and print a message to the console.
#                     matched_words.append(word) # add the word to the list of matched words, if it matches the regex pattern
#         return matched_words, patterns
#     except Exception as e:
#         print(e)
#         return [],[]


In [5]:
blacklist = ['fuck','cunt','rape','kill']

In [7]:

# example: usage example,
# upvotes: number of upvotes on Urban Dictionary,
# downvotes: number of downvotes on Urban Dictionary
import time
import random
from ratelimit import limits, sleep_and_retry
from tqdm import tqdm
# import RateLimitException from UrbanDictionary

wikitest = False # set to true to test the wikipedia page length
# include a phrase if it has a combined total of at least 100 upvotes and downvotes on Urban Dictionary
rand_dict = {}
total_votes_thresh = 200 # min number of upvotes + downvotes
upvotes_thresh = 100 # min number of upvotes
downvotes_thresh = total_votes_thresh//4 # maximum number of downvotes
desired_number_of_cards = 50
rands = [] # the randoms

@limits(calls=1, period=10) # limit the number of calls to 1 per ten seconds
def rate_limited_retrieval():
  # create a list of words that are not allowed in the definitions of the cards
  rand = ud.random() # returns a list of 5 random phrases and definitions from Urban Dictionary
  return rand

while len(rand_dict) < desired_number_of_cards:
  # get a list of 5 random phrases and definitions from Urban Dictionary
  try:
    rand = rate_limited_retrieval() # returns a list of 5 random phrases and definitions from Urban Dictionary
    # print('success')
  except Exception as e:
    #*print('Rate limit exceeded or this error: {}. \nWaiting 10 seconds...'.format(e))
    time.sleep(random.randint(5,10))
    continue
  #*print(f'Got {len(rand)} random phrases from Urban Dictionary')
  # append these to a master list
  rands.extend(rand) # rands is a list of all the random phrases and definitions from Urban Dictionary

  # iterate over the elements in the rand object
  for element in rand:
    # extract the relevant data from the element
    phrase = element.word
    # check for any blacklist word in phrase.lower() and if so, skip
    if any(word in phrase.lower() for word in blacklist):
      print(phrase, ' *#$!@#')
      continue
    definition = element.definition
    usage_example = element.example
    upvotes = element.upvotes
    downvotes = element.downvotes
    print(f'{phrase}: up({upvotes}) down({downvotes})', end='')
    # define a list of boolean values
    values = [upvotes + downvotes >= total_votes_thresh,
              upvotes >= upvotes_thresh and downvotes <= downvotes_thresh,
              check_for_good_patterns(definition, phrase),
              (upvotes > downvotes and phrase.find(' ') > 0)]
    #!print(phrase, ':', definition[0:30], '...')

    # Go ahead and save the phrase and definition if the phrase and definition are not already in the dictionary, we will filter later.
    if phrase in rand_dict.keys(): # check if the phrase is already in the dictionary
      print('-')
      continue
    else: # if the phrase is not in the dictionary, add it, and unpack the definition
      rand_dict[phrase] = unpack_definitions(phrase,definition)[1] # remove brackets and clean up the definitions with regex
      print(' +++  Added to dictionary')

    #*print(f'{phrase}: {definition}...')
    #^ Phase 2 (filtering)
    # check if any element in the list is True
    if any(values) and not check_for_badwords(definition, bad_patterns) and not check_for_badwords(phrase, bad_patterns): # check if the phrase or definition contains any buzzwords
      # include the element in the dictionary if it has a combined total of at least 100 upvotes and downvotes on Urban Dictionary
      definition = remove_undesireable_sentences(definition, phrase, bad_patterns) # remove sentences that contain buzzwords, are not in English, etc.
      definition = unpack_definitions(phrase,definition) # remove brackets and clean up the definitions with regex

      print(f'-- Added {phrase}: {definition}...')
      rand_dict[phrase] = definition


# print the number of phrases in the dictionary
print(f'Number of phrases in the dictionary: {len(rand_dict)}')
import json
# save the dictionary to a json file
with open('cards.json', 'w') as f:
  json.dump(rand_dict, f)



#! Phase 3 (filtering)

# open the latest cards.json file and load it into a dictionary
import json
with open('cards.json', 'r') as f:
    cards = json.load(f)

# convert to pandas dataframe
df = pd.DataFrame.from_dict(cards, orient='index')
# convert the index to a column
df.reset_index(inplace=True)
# rename the columns
df.columns = ['phrase', 'definition']

# now go through every phrase, and definition and replace instances of any keys that exist in the replacements dict with the corresponding value.
for i in range(len(df)):
    phrase = df.loc[i, 'phrase']
    definition = df.loc[i, 'definition']
    phrase, definition = replacer_censor(definition, phrase, replacements)
    df.loc[i, 'phrase'] = phrase
    df.loc[i, 'definition'] = definition

# convert the dataframe back to a dictionary
cards = df.set_index('phrase').T.to_dict('list')

# save the dictionary to a json file
with open('cards.json', 'w') as f:
    json.dump(cards, f, indent=4)

view: up(58) down(35) +++  Added to dictionary
SWAX: up(4) down(8) +++  Added to dictionary
penis pilgrim: up(5) down(1) +++  Added to dictionary
Gelfmania: up(5) down(0) +++  Added to dictionary
CMB: up(20) down(47) +++  Added to dictionary
Glumagen: up(2) down(0) +++  Added to dictionary
Armchair english teacher: up(1) down(1) +++  Added to dictionary
Loserville: up(58) down(20) +++  Added to dictionary
Cuban Sunrise: up(20) down(13) +++  Added to dictionary
Young bahhs: up(2) down(1) +++  Added to dictionary
2tooty2: up(2) down(1) +++  Added to dictionary
Béla: up(114) down(44) +++  Added to dictionary
HYBB: up(7) down(15) +++  Added to dictionary
hot 20: up(7) down(0) +++  Added to dictionary
Pluralis: up(1) down(1) +++  Added to dictionary
urp: up(9) down(31) +++  Added to dictionary
forbidden definitions: up(3) down(3) +++  Added to dictionary
*#$!@#
Rick'd up: up(1) down(1) +++  Added to dictionary
Handle: up(60) down(57) +++  Added to dictionary
Shot up doubles: up(1) down(0) +

In [ ]:
print(f'Number of phrases in the dictionary: {len(rand_dict)}')
import json
# save the dictionary to a json file
with open('cards.json', 'w') as f:
  json.dump(rand_dict, f)


In [ ]:
# # open the latest cards.json file and load it into a dictionary
# import json
# with open('cards.json', 'r') as f:
#     cards = json.load(f)

# # convert to pandas dataframe
# df = pd.DataFrame.from_dict(cards, orient='index')
# # convert the index to a column
# df.reset_index(inplace=True)
# # rename the columns
# df.columns = ['phrase', 'definition']

# # now go through every phrase, and definition and replace instances of any keys that exist in the replacements dict with the corresponding value.
# for i in range(len(df)):
#     phrase = df.loc[i, 'phrase']
#     definition = df.loc[i, 'definition']
#     phrase, definition = replacer_censor(definition, phrase, replacements)
#     df.loc[i, 'phrase'] = phrase
#     df.loc[i, 'definition'] = definition

# # convert the dataframe back to a dictionary
# cards = df.set_index('phrase').T.to_dict('list')

# # save the dictionary to a json file
# with open('cards.json', 'w') as f:
#     json.dump(cards, f, indent=4)

In [ ]:

# save the cards to a csv file
import pandas as pd
df = pd.DataFrame.from_dict(rand_dict, orient='index')
df.to_csv('cards.csv', header=False)

# save the cards to a json file
import json
import datetime

try:
  with open('cards_{}.json'.format(str(datetime.datetime.now())), 'w') as f:
      json.dump(rand_dict, f)
except Exception as e:
  print(e)

  with open('saved_cards.json', 'w') as f:
      json.dump(rand_dict, f)

  
# Hippopotomonstrosesquippedaliophobia - fear of long words

In [ ]:
from nltk.tokenize import word_tokenize

#todo --- test this before using it
def check_description_for_leaks(description, phrase):
    # check if the description contains any of the words in the phrase, if so change the words to similar words, and return the new description using the same capitalization as the original description, and nltk to tokenize the words.
    # tokenize the description and phrase
    description_words = word_tokenize(description)
    phrase_words = word_tokenize(phrase)

    # loop through the phrase words and check if they are in the description
    for i, word in enumerate(description_words):
        if word in phrase_words:
            # replace the word with a similar word
            description_words[i] = 'XXXX'

    # join the modified words back into a single string
    modified_description = ' '.join(description_words)

    # loop through the phrase words and modify the capitalization in the modified description
    for word in phrase_words:
        # find the index of the original word in the description
        index = description.find(word)
        # get the capitalization of the original word
        capitalization = description[index:index+len(word)]
        # modify the capitalization of the XXXX string
        if capitalization.isupper():
            modified_word = 'XXXX'.upper()
        elif capitalization.istitle():
            modified_word = 'XXXX'.title()
        else:
            modified_word = 'XXXX'
        # replace the original word with the modified XXXX string
        modified_description = modified_description.replace(word, modified_word, 1)

    return modified_description


def check_for_badwords(definition, bad_patterns):
    # if any of the buzzwords are found return a list of the matches, else an empty list
    # include porter stemmer to match variations of the words in the bad_patterns list
    from nltk.stem import PorterStemmer
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in bad_patterns if len(stemmer.stem(word)) >= 4]
    # only include the stemmed words which begin with the same letter as the original word (this is to avoid adding words that are not related to the original word)
    stemmed_words = [word for word in stemmed_words if word[0] == word[0]]
    bad_patterns.extend(stemmed_words)
    # only include stemmed words that are not already in the list of words
    bad_patterns = list(set(bad_patterns))
    # create a list of regex patterns that match the words in the list
    patterns = [re.compile(rf'\b{word}\b', re.IGNORECASE) for word in bad_patterns]
    # return the list of regex patterns (regex patterns are used to match words, and are more efficient than using the 'in' operator)
    
    matches = [match.group(0) for match in re.finditer(r'\b(?:' + '|'.join(bad_patterns) + r')\b', definition, re.IGNORECASE)]
    print('Buzzwords found:', matches)
    return matches


In [ ]:
# Testing the patterns
test = "a racial slur used by nigga ignorant hypocritical right wing retards. if you think about it, we are all illegals, the ancestors came sex into north america the same way that mexicans are entering into our country and becoming citizens now. so if you have a problem with this, then you are a racist hypocrite!"

check_for_badwords(test, bad_patterns=bad_patterns)

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

def has_sexual_comment(phrase, bad_patterns):
    # analyze the sentiment of the phrase
    scores = analyzer.polarity_scores(phrase)
    # check if the compound score is greater than or equal to 0.5, which indicates a positive sentiment
    if scores['compound'] <= 0.5:
        print(f' * Found a negative sentiment in the phrase: {phrase}')
        print(f'Score: {scores}')
    
        # if the phrase also contains a buzzword, return True
        if any(re.match(pattern, phrase) for pattern in bad_patterns):
            print(f' ** Also, Found a bad pattern in the phrase: {phrase}')
            return True
        else:
            return False
    else:
        return False


In [ ]:
test_phrase = "a single or collection of females that you shamefully solicit for sexual acts but what be disparaged if anyone knew that you were hooking up with them."

has_sex = has_sexual_comment(test_phrase, bad_patterns)

print('Testing for sex in the phrase, {}: {}'.format(test_phrase,has_sex))

In [ ]:
test_phrase =  "another phrased coined by david alan weinshel wheeler 02 bu 06. simplifies the stress of saying how and come separately. also eliminates the need to hit hte spacebar when typing."
has_sex = has_sexual_comment(test_phrase, bad_patterns)

print('Testing for sex in the phrase, {}: {}'.format(test_phrase,has_sex))

In [ ]:
real_time_trending_searches_dict

In [ ]:
import requests
import random
import re
from bs4 import BeautifulSoup
from datetime import datetime
from PyDictionary import PyDictionary
import wikipedia

dictionary = PyDictionary() # instantiate 

def get_wiki_definition(card):
  # Get the summary of the Wikipedia page
  summary = wikipedia.summary(card)
  return summary

# # Example usage:
# card = "John Smith"
# definition = get_definition(card)
# print(definition)  # prints the summary of the Wikipedia page "John Smith"

def get_definition(card):
  # Check the type of the card
  if is_slang_phrase(card):
    # Get the definition of the slang phrase from Urban Dictionary
    url = "https://api.urbandictionary.com/v0/define?term=" + card
    response = requests.get(url)
    data = response.json()
    definition = data['list'][0]['definition']
  elif is_person(card):
    # Get the summary of the person from Wikipedia
    definition = get_wiki_definition(card[0]) # for a card title get the definition (summary) of the wikipedia page.
  else:
    # this is a word so use PyDictionary library.
    definition = dictionary.meaning(str(card[0])) # this could have mult. meanings returned. 
    definition = definition[0] # the first meaning. 

def get_random_slang_phrase():
  # Make a request to the Urban Dictionary API to get a random slang phrase
  url = "https://api.urbandictionary.com/v0/random"
  response = requests.get(url)
  data = response.json()
  slang_phrase = data['list'][0]['word']
  return slang_phrase

def get_random_wiki_person():
  # Make a request to Wikipedia API to get a random person
  url = "https://en.wikipedia.org/w/api.php?action=query&format=json&list=random&rnnamespace=0&rnlimit=1"
  response = requests.get(url)
  data = response.json()
  person = data['query']['random'][0]['title']
  return person

def get_random_wiki_word():
  # Make a request to Wikipedia API to get a random word
  url = "https://en.wikipedia.org/w/api.php?action=query&format=json&list=random&rnnamespace=0&rnlimit=1"
  response = requests.get(url)
  data = response.json()
  word = data['query']['random'][0]['title']
  return word

def create_a_card_deck(num_cards=30):
  # create 30 random cards with equal distributions of slang, people, and words.
  # avoid profanity, or lude sexual topics.
  
  # Create a list of all the cards
  cards = []

  # Initialize a dictionary to keep track of the count of each type of card
  counts = {
    "slang": 0,
    "people": 0,
    "words": 0
  }

  # While there are fewer than 30 cards in the deck, generate a new card
  while len(cards) < num_cards:
    # Find the type of card with the lowest count
    min_count_type = min(counts, key=counts.get)

    # Generate a new card of that type
    if min_count_type == "slang":
      slang_phrase = get_random_slang_phrase()
      new_card = (slang_phrase, "slang", get_definition(slang_phrase))
    elif min_count_type == "people":
      person = get_random_wiki_person()
      new_card = (person, "people", get_definition(person))
    else:
      word = get_random_wiki_word()
      new_card = (word, "word", get_definition(word))

    # Add the new card to the list and update the count
    cards.append(new_card)
    counts[min_count_type] += 1 
  return cards

def is_slang_phrase(card):
  if card[1] == 'slang':
    return True
  else:
    return False

def is_person(card):
  if card[1] == 'person':
    return True
  else:
    return False

def filter_cards(cards):
  # create a function to filter out inappropriate content
  filtered_cards = []
  for card in cards:

    # check if the card contains profanity or sexual content
    print("checking card: {}".format(card))
    definition = card[2] # definition
    card_type = card[1] # i.e. 'slang'
    word = card[0] # word 
    try:
      if "profanity" in definition or "sex" in definition or "lewd" in definition:
        continue
      elif check_for_badwords():
        continue
      else:
        filtered_cards.append(card)
    except Exception as e:
      print(f'error with finding the meaning in the soup object. soup comes from the urban dictionary page.')
      print(e)
  return filtered_cards

# generate a deck of cards
card_deck = create_a_card_deck()

# filter the deck for inappropriate content
filtered_deck = filter_cards(card_deck)

# print the cards
print('Generated {} cards'.format(len(filtered_deck)))

In [ ]:
# print the filtered cards
for card in filtered_deck:
  print(card)


In [ ]:
# Example usage:
card = "dope"
definition = get_definition(card)
card_type = "phrase"
card_preview = 

# (slang_phrase, "slang", get_definition(slang_phrase))
    


print(definition)  # prints the definition of the slang phrase "dope"

card = "John Smith"
definition = get_definition(card)
print(definition)  # prints the summary of the person "John Smith"

card = "tree"
definition = get_definition(card)
print(definition)  # prints the definition of the word "tree"
